## MLflow's Model Registry

In [1]:
import os
import mlflow
from mlflow.tracking import MlflowClient
from dotenv import find_dotenv, load_dotenv

# upload env variables from .env-file in our work environment
load_dotenv(find_dotenv())

mlflow_server_uri = os.getenv("MLFLOW_SERVER_URI")
mlflow.set_tracking_uri(mlflow_server_uri)

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=mlflow_server_uri)

In [3]:
client.create_experiment(name="my-cool-experiment")

'22'

Let's check the latest versions for the experiment with id `1`...

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='17',
    filter_string="metrics.rmse < 2.47",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    # print(run)
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 8ab25be8e16045628e2bbadba2058761, rmse: 2.4498
run id: 575c0e9109684e71b6df957db920772d, rmse: 2.4514
run id: 69ae79284195494ea9a0cfe5c3371db3, rmse: 2.4516
run id: 73ba2a3c09224517a3c3b688677f627f, rmse: 2.4539
run id: 0c4d84aa85b5418c8bf97e977135491c, rmse: 2.4560


In [7]:
run

<Run: data=<RunData: metrics={'rmse': 2.455971238567075}, params={'max_depth': '19',
 'min_samples_leaf': '4',
 'min_samples_split': '10',
 'n_estimators': '12',
 'n_jobs': '-1',
 'random_state': '42'}, tags={'mlflow.runName': 'rebellious-skink-515',
 'mlflow.source.git.commit': '7a1e1bad72bcda019f752e30599de6dc6ef9ba74',
 'mlflow.source.name': 'hpo.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'admin-gpu'}>, info=<RunInfo: artifact_uri='s3://mlflow-bucket/17/0c4d84aa85b5418c8bf97e977135491c/artifacts', end_time=1685628526750, experiment_id='17', lifecycle_stage='active', run_id='0c4d84aa85b5418c8bf97e977135491c', run_name='rebellious-skink-515', run_uuid='0c4d84aa85b5418c8bf97e977135491c', start_time=1685628526466, status='FINISHED', user_id='admin-gpu'>>

### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [8]:
import mlflow

mlflow.set_tracking_uri(mlflow_server_uri)

In [9]:
run_id = "8ab25be8e16045628e2bbadba2058761"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="ni-taxi-reg")

Registered model 'ni-taxi-reg' already exists. Creating a new version of this model...
2023/06/02 08:45:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ni-taxi-reg, version 3
Created version '3' of model 'ni-taxi-reg'.


<ModelVersion: aliases=[], creation_timestamp=1685695551734, current_stage='None', description='', last_updated_timestamp=1685695551734, name='ni-taxi-reg', run_id='8ab25be8e16045628e2bbadba2058761', run_link='', source='s3://mlflow-bucket/17/8ab25be8e16045628e2bbadba2058761/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [12]:
model_name = "ni-taxi-reg"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 3, stage: None


In [11]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685692266101, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2023-06-02', last_updated_timestamp=1685695558292, name='ni-taxi-reg', run_id='a282dbac4c514a5a9163cb76ad577e1e', run_link='', source='s3://mlflow-bucket/14/a282dbac4c514a5a9163cb76ad577e1e/artifacts/models_mlflow', status='READY', status_message='', tags={}, user_id='', version='1'>

In [13]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685692266101, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2023-06-02', last_updated_timestamp=1685695570177, name='ni-taxi-reg', run_id='a282dbac4c514a5a9163cb76ad577e1e', run_link='', source='s3://mlflow-bucket/14/a282dbac4c514a5a9163cb76ad577e1e/artifacts/models_mlflow', status='READY', status_message='', tags={}, user_id='', version='1'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [16]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [17]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [ ]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [39]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

In [41]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 139 ms, sys: 44.6 ms, total: 183 ms
Wall time: 447 ms


{'rmse': 6.659623830022514}

In [42]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.94 s, sys: 216 ms, total: 7.16 s
Wall time: 7.28 s


{'rmse': 6.881555517147188}

In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>